# Split Video features into app and mot

In [ ]:
import torch
import numpy as np
import h5py

train_datapath = "/home/adithyas/NExT-QA/data/feats/vid_feat/app_mot_train.h5"
val_datapath = "/home/adithyas/NExT-QA/data/feats/vid_feat/app_mot_val.h5"
test_datapath = "/home/adithyas/NExT-QA/data/feats/vid_feat/app_mot_test.h5"

In [ ]:
def load_data(features_path='app_mot_train.h5', id_key="ids", features_key="feat"):
    with h5py.File(features_path, 'r') as f:
        # (3870,) # (3870, 16, 4096)
        ids, features = f[id_key][()], f[features_key][()]
    return ids, features

In [3]:
id_key, features_key = "ids", "feat"
train_ids, train_features = load_data(train_datapath, id_key, features_key)
val_ids, val_features = load_data(val_datapath, id_key, features_key)
test_ids, test_features = load_data(test_datapath, id_key, features_key)
# (3870, 16, 4096) (570, 16, 4096) (1000, 16, 4096)
print(f"{train_features.shape} {val_features.shape} {test_features.shape}")

(3870, 16, 4096) (570, 16, 4096) (1000, 16, 4096)


In [28]:
train_app_features, train_motion_features = train_features[:, :, :2048], train_features[:, :, 2048:]
# (3870, 16, 2048) (3870, 16, 2048)
print(f"{train_app_features.shape} {train_motion_features.shape}")

val_app_features, val_motion_features = val_features[:, :, :2048], val_features[:, :, 2048:]
test_app_features, test_motion_features = test_features[:, :, :2048], test_features[:, :, 2048:]

(3870, 16, 2048) (3870, 16, 2048)


In [29]:
train = h5py.File('/home/adithyas/NExT-QA/data/feats/vid_feat/train_features.h5', 'w')
val = h5py.File('/home/adithyas/NExT-QA/data/feats/vid_feat/val_features.h5', 'w')
test = h5py.File('/home/adithyas/NExT-QA/data/feats/vid_feat/test_features.h5', 'w')

train.create_dataset('app_feat', data=train_app_features)
train.create_dataset('mot_feat', data=train_motion_features)

val.create_dataset('app_feat', data=val_app_features)
val.create_dataset('mot_feat', data=val_motion_features)

test.create_dataset('app_feat', data=test_app_features)
test.create_dataset('mot_feat', data=test_motion_features)

train.close()
val.close()
test.close()

# Load model weights

In [4]:
from networks.hga import HGA
data_dict = torch.load("/home/adithyas/IGV/weights/hga_model.ckpt")
model = HGA(1852, 512,  512, 0.5, 1, 1, memory=train_features)
model.load_state_dict(data_dict)

<All keys matched successfully>